<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D1%8C_%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7_%D0%BC%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# convlstm model for marketing
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split


In [2]:

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=0, sep=';', dtype = {'ЗакупочнаяЦена': np.float32, 'РегулярнаяЦена': np.float32, 'ПродажиНеделя': np.float32, 'АкцияСкидка': np.float32})
	return dataframe


In [3]:
df = load_file('ДанныеПродаж.csv')
df['id'] = df['НаименованиеОЛАП_id'] + '#'+df['Контрагент_id']+ '#'+df['ДоговорКонтрагента_id']
df['СредниеПродажи'] = df['ПродажиНеделя'] / (df['Рабочий'] + df['Предпраздничный'] + df['Суббота'] + df['Праздник'] + df['Воскресенье']) 
#df = df.loc[df['Контрагент_id'] == 'b65fd636-4af4-11df-ae49-0030487d100c']
df = df.drop(['НаименованиеОЛАП', 'НаименованиеОЛАП_id', 'ГруппаДоговоров', 'ГруппаДоговоров_id', 'ДоговорКонтрагента', 'ДоговорКонтрагента_id', 'Контрагент', 'Контрагент_id', 'ПродажиНеделя'],axis = 1)
df['por'] = df['Год']*10000 + df['Месяц']*100+ df['Неделя']
df['price'] = df['РегулярнаяЦена']/df['ЗакупочнаяЦена']
df = df.drop(['Неделя','Месяц','Год','РегулярнаяЦена', 'ЗакупочнаяЦена'] ,axis = 1)

In [4]:
df.shape

(95096, 10)

In [8]:
df.head(10)

,Рабочий,Предпраздничный,Суббота,Воскресенье,Праздник,АкцияСкидка,id,СредниеПродажи,por,price
0,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173
1,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173
2,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#ee785bd2-...,6.666667,20190101,1.170889
3,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#2a8b5a65-...,2.500000,20190101,1.064052
4,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173
5,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173
6,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#ee785bd2-...,3.333333,20190101,1.170889
7,0,0,0,1,5,0.15,0d7f65f2-4eb8-11e8-8103-00155d3c153d#64871255-...,0.715000,20190101,1.262815
8,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173
9,0,0,0,1,5,0.00,0d7f65f2-4eb8-11e8-8103-00155d3c153d#b65fd636-...,0.000000,20190101,0.985173


In [31]:
def data_convert(df, step = 128):
  sp = df.id.unique()
  X = list()
  Y = list()
    
  for el in sp:
    spl = df.loc[df['id'] == el]
    spl.sort_values(by=['por'], inplace=True)
    #print(spl)
    x = np.array(spl[['СредниеПродажи','Рабочий','Предпраздничный','Суббота','Праздник','Воскресенье','АкцияСкидка','price']])
    z = np.array(spl[['Рабочий','Предпраздничный','Суббота','Праздник','Воскресенье','АкцияСкидка','price']])
    y = np.array(spl[['СредниеПродажи']])
    #print (x[-6:], y[-10:])
    for i in range(5, x.shape[0]-step):
      st = np.concatenate(( x[i-5:i+step-5], z[i-4:i+step-4] , z[i-3:i+step-3],z[i-2:i+step-2],z[i-1:i+step-1],z[i:i+step] ), axis=1)
      X.append(st)
      st = np.concatenate((y[i+step-4],y[i+step-3],y[i+step-2],y[i+step-1],y[i+step]), axis=0)
      Y.append(st)
    #return Y, X  
      
      
  return train_test_split(np.asarray(X).astype(np.float32), np.asarray(Y).astype(np.float32), test_size=0.2, random_state=314)


In [95]:


# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, MName):
	# define model
  verbose, epochs, batch_size = 1, 100, 25
  model = Sequential()
  
  if MName == 'LSTM':
  # для модели LSTM
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='relu'))
  
  elif MName == 'CNNLSTM':
  # для модели CNNLSTM точность -   
  # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_features, n_outputs = trainX.shape[2], trainy.shape[1]
    n_steps = 4
    n_length = int(trainX.shape[1]/n_steps)
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model.add(TimeDistributed(Conv1D(64, 3, activation='relu'), input_shape=(None, n_length,n_features)))
    model.add(TimeDistributed(Conv1D(64, 3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D()))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, input_shape=(n_steps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='relu')) 
  
  elif MName == 'ConvLSTM2D':
    # Модель ConvLSTM2D - 64% точности
    n_features, n_outputs = trainX.shape[2], trainy.shape[1]
    n_steps = 4
    n_length = int(trainX.shape[1]/n_steps)
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
    model.add(ConvLSTM2D(64, (1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='relu'))
  
  
  
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MAE'])
  print(model.summary())
  # fit network
  #model.load_weights('marketing_5week_weights')
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  model.save_weights('marketing_5week_weights')
	# evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=verbose)
  return accuracy, model, _

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(df, MName, repeats=1):
  # load data
  trainX, testX, trainy, testy = data_convert(df)
	# repeat experiment
  scores = list()
  for r in range(repeats):
	  score, model, result = evaluate_model(trainX, trainy, testX, testy, MName)
	  score = 100 - score / np.mean(testy) * 100
	  print('>#%d: %.3f' % (r+1, score))
	  scores.append(score)
	# summarize results
  summarize_results(scores)
  return model

# run the experiment


In [96]:
model = run_experiment(df, 'ConvLSTM2D')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_9 (ConvLSTM2D)  (None, 1, 30, 64)         82432     
_________________________________________________________________
dropout_36 (Dropout)         (None, 1, 30, 64)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 100)               192100    
_________________________________________________________________
dense_41 (Dense)             (None, 5)                 505       
Total params: 275,037
Trainable params: 275,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100


509/509 [==============================] - 45s 23ms/step - loss: 14963.3643 - MAE: 9.0609
Epoch 2/100
509/509 [==============================] - 12s 23ms/step - loss: 12593.1592 - MAE: 8.3606
Epoch 3/100
509/509 [==============================] - 12s 23ms/step - loss: 11124.3916 - MAE: 8.1495
Epoch 4/100
509/509 [==============================] - 11s 23ms/step - loss: 10947.5908 - MAE: 8.6264
Epoch 5/100
509/509 [==============================] - 12s 23ms/step - loss: 9960.4375 - MAE: 8.2517
Epoch 6/100
509/509 [==============================] - 11s 23ms/step - loss: 8375.8457 - MAE: 7.9736
Epoch 7/100
509/509 [==============================] - 11s 22ms/step - loss: 9512.6758 - MAE: 8.3858
Epoch 8/100
509/509 [==============================] - 11s 22ms/step - loss: 10349.6992 - MAE: 8.5252
Epoch 9/100
509/509 [==============================] - 11s 22ms/step - loss: 9103.1035 - MAE: 8.0912
Epoch 10/100
509/509 [==============================] - 11s 22ms/step - loss: 6823.2139 - MAE: 7.3

In [83]:
model.save_weights('marketing_5week_weights')
model.save('marketing_5week')

INFO:tensorflow:Assets written to: marketing_5week/assets


INFO:tensorflow:Assets written to: marketing_5week/assets


In [40]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_5 (ConvLSTM2D)  (None, 1, 30, 64)         82432     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 30, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               192100    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 505       
Total params: 275,037
Trainable params: 275,037
Non-trainable params: 0
_________________________________________________________________


In [ ]:
trainX, testX, trainy, testy = data_convert(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def predict(testX, testy, model):
	# define model
  n_features, n_outputs = testX.shape[2], testy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 4, int(trainX.shape[1]/4)
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	
  return model.predict(testX)

In [ ]:
predY = predict(testX, testy, model)

In [ ]:
x = testX[0]

In [ ]:
x[0]

array([0.14285715, 5.        , 0.        , 1.        , 0.        ,
       1.        , 0.15      , 5.        , 0.        , 1.        ,
       0.        , 1.        , 0.15      , 5.        , 0.        ,
       1.        , 0.        , 1.        , 0.06      , 2.        ,
       0.        , 1.        , 0.        , 1.        , 0.        ,
       3.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 5.        , 0.        , 1.        , 0.        ,
       1.        , 0.        ], dtype=float32)

In [ ]:
def res(x, ind):
  for j in range(0,10):
    print(x[ind][j][0], x[ind][j][6])

In [ ]:
res(testX, 150) , predY[150]

0.28285715 0.0
0.535 0.0
0.71333337 0.0
0.51 0.15
0.61285716 0.15
0.95857143 0.06
0.23857142 0.0
0.23857142 0.0
0.48714286 0.0
0.2857143 0.0


(None, array([0.2461024 , 0.25892362, 0.21842286, 0.75727165, 0.16739061],
       dtype=float32))